In [ ]:
import snowflake.snowpark as snowpark

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
DB_NAME = "CHUNKING_EVAL_DEMO"
SCHEMA_NAME = "DATA"
STAGE_NAME = "DOCS"
WH_NAME = "CHUNKING_EVAL_WAREHOUSE"

In [ ]:
-- List files in the stage to identify PDFs
LS @{{DB_NAME}}.{{SCHEMA_NAME}}.{{STAGE_NAME}}

In [ ]:
import os

#Define quick function to add local file to snowflake stage
def upload_file_to_stage(filename: str, target_stage: str):
    print(f"Adding file {filename} to {target_stage} stage...")
    put_result =  session.file.put(
        local_file_name = f"pdfs/{filename}",
        stage_location =  f'@"{target_stage}"',
        auto_compress = False,
        source_compression= 'AUTO_DETECT',
        overwrite = True)
    return put_result


local_pdfs = []
for root, dirs, files  in os.walk('pdfs/'):
    for file in files:
        if file.endswith('.pdf'):
            local_pdfs.append(file)


#Get list of filenames already in stage
stage_pdfs = [row[0][row[0].find('/')+1:] for row in session.sql(f'LS @{DB_NAME}.{SCHEMA_NAME}.{STAGE_NAME}').select('"name"').collect()]

for pdf in local_pdfs:
    if pdf in stage_pdfs:
        print(f"File {pdf} already in {STAGE_NAME} stage!")
    else:
        upload_file_to_stage(pdf, STAGE_NAME)

## Step 1: Parse and Chunk Text from PDFs
We begin by parsing the content of uploaded PDFs and chunking the text using Snowflake's [PARSED_TEXT](https://docs.snowflake.com/sql-reference/functions/parse_document-snowflake-cortex) and [SPLIT_TEXT_RECURSIVE_CHARACTER](https://docs.snowflake.com/sql-reference/functions/split_text_recursive_character-snowflake-cortex) features. These steps structure the text into manageable segments optimized for retrieval. To ensure that the PDF parsing and chunking have been processed correctly, we run queries on the parsed and chunked tables. This step helps verify the integrity of the content.

Objective: **Transform unstructured content into indexed chunks for efficient search and retrieval.**

Key Outputs:
- SKO.HOP.PARSED_TEXT: Table containing the raw text.
- SKO.HOP.CORTEX_CHUNK: Chunked, searchable content.

In [1]:
-- Create a table to hold the extracted text from the PDF files
CREATE OR REPLACE TABLE {{DB_NAME}}.{{SCHEMA_NAME}}.PARSED_TEXT (relative_path VARCHAR(500), raw_text VARIANT);

SyntaxError: invalid syntax (579301450.py, line 1)

In [ ]:
INSERT INTO {{DB_NAME}}.{{SCHEMA_NAME}}.PARSED_TEXT (relative_path, raw_text)
WITH pdf_files AS (
    SELECT DISTINCT
        METADATA$FILENAME AS relative_path
    FROM @{{DB_NAME}}.{{SCHEMA_NAME}}.DOCS
    WHERE METADATA$FILENAME ILIKE '%.pdf'
      -- Exclude files that have already been parsed
      AND METADATA$FILENAME NOT IN (SELECT relative_path FROM PARSED_TEXT)
)
SELECT 
    relative_path,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        '@{{DB_NAME}}.{{SCHEMA_NAME}}.DOCS',  -- Your stage name
        relative_path,  -- File path
        {'mode': 'layout'}  -- Adjust mode as needed ('layout', 'ocr')
    ) AS raw_text
FROM pdf_files;

In [ ]:
-- inspect the results and count the tokens for each document
SELECT *, SNOWFLAKE.CORTEX.COUNT_TOKENS('mistral-7b', RAW_TEXT) as token_count
FROM {{DB_NAME}}.{{SCHEMA_NAME}}.PARSED_TEXT;

In [ ]:
ALTER TABLE PARSED_TEXT 
    ADD COLUMN IF NOT EXISTS DOC_SUMMARY VARCHAR(5000);


UPDATE PARSED_TEXT 
  SET DOC_SUMMARY = AI_COMPLETE('claude-4-sonnet', CONCAT('Concisely summarize the following text of meeting minutes', RAW_TEXT))

In [ ]:
-- Chunk the text based on paragraph seperators and write into DOC_CHUNKS_TABLE;

CREATE OR REPLACE TABLE {{DB_NAME}}.{{SCHEMA_NAME}}.PARAGRAPH_CHUNKS AS
WITH text_chunks AS (
    SELECT
        relative_path,
        doc_summary,
        SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
            raw_text:content::STRING,  -- Extract the 'content' field from the JSON
            'markdown', -- Adjust to 'markdown' if needed
            2000,       -- Adjust chunk size
            100,        -- Adjust overlap size
            ['\n\n']    -- Adjust separators
        ) AS chunks
    FROM {{DB_NAME}}.{{SCHEMA_NAME}}.PARSED_TEXT
)
SELECT
    relative_path,
    doc_summary,
    c.value AS chunk  -- Extract each chunk of the parsed text
FROM text_chunks,
LATERAL FLATTEN(INPUT => chunks) c;

In [ ]:
-- Check the results and 

SELECT *, SNOWFLAKE.CORTEX.COUNT_TOKENS('mistral-7b', CHUNK) as token_count
FROM {{DB_NAME}}.{{SCHEMA_NAME}}.PARAGRAPH_CHUNKS;

In [ ]:
ALTER TABLE PARAGRAPH_CHUNKS ADD COLUMN IF NOT EXISTS MEETING_YEAR VARCHAR(4);
ALTER TABLE PARAGRAPH_CHUNKS ADD COLUMN IF NOT EXISTS MEETING_MONTH VARCHAR(2);
ALTER TABLE PARAGRAPH_CHUNKS ADD COLUMN IF NOT EXISTS ABSOLUTE_PATH VARCHAR(500);


UPDATE PARAGRAPH_CHUNKS 
  SET MEETING_YEAR = SUBSTRING(RELATIVE_PATH, 14, 4),
      MEETING_MONTH = SUBSTRING(RELATIVE_PATH, 19, 2),
      ABSOLUTE_PATH =  GET_ABSOLUTE_PATH('@DOCS', CONCAT('docs/', RELATIVE_PATH));

In [ ]:
ALTER TABLE PARAGRAPH_CHUNKS
    ADD COLUMN TEMP_COL VARCHAR;

UPDATE PARAGRAPH_CHUNKS
    SET TEMP_COL = CAST(CHUNK AS VARCHAR);

ALTER TABLE PARAGRAPH_CHUNKS
    DROP COLUMN CHUNK;
    
ALTER TABLE PARAGRAPH_CHUNKS
    RENAME COLUMN TEMP_COL TO CHUNK;

In [ ]:
ALTER TABLE PARAGRAPH_CHUNKS ADD COLUMN IF NOT EXISTS CHUNK_WITH_SUMMARY VARCHAR;


UPDATE PARAGRAPH_CHUNKS 
  SET CHUNK_WITH_SUMMARY = DOC_SUMMARY || '\n\n' || CHUNK

In [ ]:
-- Create a search service over your new chunked pdf table

CREATE OR REPLACE CORTEX SEARCH SERVICE {{DB_NAME}}.{{SCHEMA_NAME}}.FOMC_RAW_TEXT_RETRIEVAL
    ON SEARCH_COL
    ATTRIBUTES MEETING_YEAR, MEETING_MONTH
    WAREHOUSE = {{WH_NAME}}
    TARGET_LAG = '1 hour'
    AS SELECT 
        ABSOLUTE_PATH,
        RELATIVE_PATH,
        CHUNK,
        MEETING_YEAR,
        MEETING_MONTH,
        CHUNK AS SEARCH_COL
    FROM {{DB_NAME}}.{{SCHEMA_NAME}}.PARAGRAPH_CHUNKS;

In [ ]:
-- Create a search service over your new chunked pdf table

CREATE OR REPLACE CORTEX SEARCH SERVICE {{DB_NAME}}.{{SCHEMA_NAME}}.FOMC_TAGGED_CHUNK_RETRIEVAL
    ON SEARCH_COL
    ATTRIBUTES MEETING_YEAR, MEETING_MONTH
    WAREHOUSE = {{WH_NAME}}
    TARGET_LAG = '1 hour'
    AS SELECT 
        ABSOLUTE_PATH,
        RELATIVE_PATH,
        CHUNK,
        MEETING_YEAR,
        MEETING_MONTH,
        CHUNK_WITH_SUMMARY as SEARCH_COL
    FROM {{DB_NAME}}.{{SCHEMA_NAME}}.PARAGRAPH_CHUNKS;

In [ ]:
# Query your Snowflake Cortex Search Service using the Snowpark Python API to retrieve and process search results.
import streamlit as st
from snowflake.snowpark import Session
from snowflake.core import Root
root = Root(session)

fomc_search_service = (root
  .databases[DB_NAME]
  .schemas[SCHEMA_NAME]
  .cortex_search_services['FOMC_TAGGED_CHUNK_RETRIEVAL']
)

resp = fomc_search_service.search(
  query="""How has global economic economy shaped interest rates in early 2024""",
  columns=['SEARCH_COL', 'RELATIVE_PATH', 'MEETING_YEAR'],
  filter={"@eq": {"MEETING_YEAR": "2024"} },
  limit=3
)
results = resp.results
results

# context_str = ""
# for i, r in enumerate(results):
#     context_str = f"Document {i+1}: \n\n {r['RELATIVE_PATH']} \n {r['CHUNK']}\n****************\n"

#     st.write(context_str)

In [ ]:
from snowflake.cortex import complete
import streamlit as st

st.write(complete('claude-4-sonnet', 
         f"""Use the provided context to answer the user question. 
         Question: How has global economic economy shaped interest rates in early 2024? 
         Context :{results[0]['SEARCH_COL']}"""))

In [ ]:
SELECT *,  SNOWFLAKE.CORTEX.COMPLETE('CLAUDE-4-SONNET', CONCAT('SUMMARIZE THE FOLLOWING TEXT SUCCINTLY AND PROVIDE A FEW QUICK SAMPLE QUESTIONS A USER MAY ASK OF THIS DOCUMENT', RAW_TEXT)) FROM PARSED_TEXT

In [ ]:
WITH chunks AS (
    SELECT
        t."RELATIVE_PATH" AS FILE_NAME,
        c.value::varchar AS chunk          -- plain‑text chunk
    FROM PARSED_TEXT AS t
    ,   LATERAL FLATTEN(
            input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
                t."RAW_TEXT",
                'markdown',
                512,       -- Adjust chunk size
                64,        -- Adjust overlap size
                ['\n\n']    -- Adjust separators
            )
        ) AS c
),

/* ----------------------------------------------------------------------
   2. One summary per document
------------------------------------------------------------------------ */
doc_summaries AS (
    SELECT
        FILE_NAME,
        AI_SUMMARIZE_AGG(chunk) AS doc_summary
    FROM chunks
    GROUP BY FILE_NAME
)

/* ----------------------------------------------------------------------
   3. Attach the summary and prepend it to each chunk
------------------------------------------------------------------------ */
SELECT
    c.FILE_NAME,
    d.doc_summary || '\n\n' || c.chunk AS chunk   -- summary + chunk
FROM chunks        AS c
JOIN doc_summaries AS d
 ON c.FILE_NAME = d.FILE_NAME;
 
--  /* ---------- contextual chunk service ---------- */
-- CREATE OR REPLACE CORTEX SEARCH SERVICE contextualized_chunk_search_svc
--   ON chunk
--   ATTRIBUTES (ID, FILE_NAME)
--   WAREHOUSE   = COMPUTE
--   TARGET_LAG  = '1 hour'
--   EMBEDDING_MODEL = 'snowflake-arctic-embed-m-v1.5'
--   AS (
--       SELECT ID, FILE_NAME, chunk
--       FROM   contextual_chunks
--   );
